In [65]:
!pip install ipython-sql
!pip install sqlalchemy


In [2]:
import pandas as pd
import sqlite3
from IPython.core.magic import register_line_magic
import camelot

## Initializing Antibiotics Database

In [67]:
# Step 1: Read the CSV file into a DataFrame
csv_file_path = 'data/antibiotics.csv'

# Try reading with UTF-8 encoding first
antibiotics_df = pd.read_csv(csv_file_path)
   
# Step 3: Remove empty columns from the DataFrame
antibiotics_df = antibiotics_df.dropna(axis=1, how='all')

# Remove unnamed columns
antibiotics_df = antibiotics_df.loc[:, ~antibiotics_df.columns.str.contains('^Unnamed')]

# sanitize column names to make them suitable for the SQL database
antibiotics_df = antibiotics_df.rename(columns={
    "ATC code": "ATC_code",
    "Listed on EML/EMLc 2023": "Listed_on_EML_EMLc_2023"
})



# Validate data (check for null values)
if antibiotics_df.isnull().values.any():
    raise ValueError("CSV file contains null values. Please clean the data before inserting.")
else:
    print('Dataframe cleaned')

Dataframe cleaned


In [68]:
antibiotics_df

,Antibiotic,Class,ATC_code,Category,Listed_on_EML_EMLc_2023
0,Amikacin,Aminoglycosides,J01GB06,Access,Yes
1,Amoxicillin,Penicillins,J01CA04,Access,Yes
2,Amoxicillin/clavulanic-acid,Beta-lactam/beta-lactamase-inhibitor,J01CR02,Access,Yes
3,Ampicillin,Penicillins,J01CA01,Access,Yes
4,Ampicillin/sulbactam,Beta-lactam/beta-lactamase-inhibitor,J01CR01,Access,No
...,...,...,...,...,...
252,Trimethoprim,Trimethoprim-derivatives,J01EA01,Access,Yes
253,Troleandomycin,Macrolides,J01FA08,Watch,No
254,Trovafloxacin,Fluoroquinolones,J01MA13,Watch,No
255,Vancomycin_IV,Glycopeptides,J01XA01,Watch,Yes


In [69]:
# Connect to SQLite database (or create it if it doesn't exist)
database_path = 'data/antibiotics.db'
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

try:
    # Step 5: Create a table in the SQLite database
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS antibiotics_table (
        {}
    )
    '''.format(', '.join(f"{col} TEXT" for col in antibiotics_df.columns))
    
    cursor.execute(create_table_query)

    # Step 6: Insert data from the cleaned DataFrame into the SQLite table
    antibiotics_df.to_sql('antibiotics_table', conn, if_exists='replace', index=False)
    conn.commit()

    # Step 7: Query the data (example query)
    query = "SELECT * FROM antibiotics_table LIMIT 5"
    result = cursor.execute(query).fetchall()

    # Display the query result
    for row in result:
        print(row)

except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()
finally:
    # Close the database connection
    conn.close()

('Amikacin', 'Aminoglycosides', 'J01GB06', 'Access', 'Yes')
('Amoxicillin', 'Penicillins', 'J01CA04', 'Access', 'Yes')
('Amoxicillin/clavulanic-acid', 'Beta-lactam/beta-lactamase-inhibitor', 'J01CR02', 'Access', 'Yes')
('Ampicillin', 'Penicillins', 'J01CA01\xa0', 'Access', 'Yes')
('Ampicillin/sulbactam', 'Beta-lactam/beta-lactamase-inhibitor', 'J01CR01', 'Access', 'No')


In [79]:
%reload_ext sql
# Step 3: Connect to the SQLite database
%sql sqlite:///data/antibiotics.db

# Step 4: Run your SQL query and display the result directly
%sql SELECT * FROM antibiotics_table LIMIT 5


 * sqlite:///data/antibiotics.db
Done.


Antibiotic,Class,ATC_code,Category,Listed_on_EML_EMLc_2023
Amikacin,Aminoglycosides,J01GB06,Access,Yes
Amoxicillin,Penicillins,J01CA04,Access,Yes
Amoxicillin/clavulanic-acid,Beta-lactam/beta-lactamase-inhibitor,J01CR02,Access,Yes
Ampicillin,Penicillins,J01CA01,Access,Yes
Ampicillin/sulbactam,Beta-lactam/beta-lactamase-inhibitor,J01CR01,Access,No


## Initializing EML Database

In [82]:
#Extract tables from EML
tables = camelot.read_pdf('data/EML.pdf',pages='1-end')
tables

<TableList n=56>

In [92]:
tables[55].df

,0,1,2,3,4,5
0,Name of Drug\nFormulation\nStrength\nLevel \nN...,,,,,
1,Sodium Bicarbonate,Injection,0.084,C,R,
2,Sodium Chloride,Injection,0.45% (250 \nml),B1,R,
3,Sodium Chloride,Injection,0.9% (250 \nml),M,NR,
4,Sodium Chloride,Injection,0.9% (500 \nml),M,R,
5,Sodium Chloride + \nPotassium Chloride,Injection,0.9% + 20 \nmMol (500 \nml),SM,NR,
6,27.3. Miscellaneous,,,,,
7,Water for Injection\nInjection\n(blank)\nA\nR,,,,,
8,28. Vitamins and Minerals,,,,,
9,Calciferol,Injection,(blank),D,R,


## ...................#TEST EXTRACTION SANDBOX..................................